# 🚀 Publish MCP Text Utilities on Azure API Management

Use this notebook to publish an already-deployed **MCP Text Utilities** Container App behind an existing
**Azure API Management** service.

## What this notebook does

1. Verifies your Azure CLI login and resolves the active subscription.
2. Deploys `apim-mcp.bicep` to the target resource group using `az deployment group create`.
   The Bicep template creates:
   - Three APIM **Named Values** (`mcp-backend-url`, `mcp-tenant-id`, `mcp-app-id`)
   - An APIM **API** at the configured path suffix
   - Three **Operations** (`POST /mcp`, `GET /sse`, `POST /messages/`)
   - An **API-level policy** (JWT validation + backend routing, loaded from `apim-policy.xml`)
3. Prints the resulting APIM endpoint URLs.

## Prerequisites

| Requirement | Notes |
|---|---|
| Azure CLI ≥ 2.50 | `az --version` |
| Bicep CLI (bundled with Azure CLI) | `az bicep version` |
| Active `az login` session | `az account show` |
| Existing APIM service | Standard or above tier recommended |
| Deployed MCP Container App | Run `azd up` from the repo root first |

## Finding required values

After running `azd up` in the repo root, the outputs are stored in `.azure/<env>/.env`:

```bash
# Example: read azd outputs
cat .azure/<env-name>/.env
# Relevant variables:
#   SERVICE_AGENT_URI          → use as mcp_server_url
#   ENTRA_API_APP_CLIENT_ID    → use as entra_api_app_client_id
#   AZURE_TENANT_ID            → use as tenant_id
```

<a id='0'></a>
### 0️⃣ Initialize notebook variables

Fill in every value marked **`# REQUIRED`** before running the remaining cells.
Optional values have sensible defaults that match the repo conventions.

In [ ]:
# ── Azure subscription & resource group ───────────────────────────────────────

subscription_id = ""           # Leave empty to use the current az subscription
resource_group  = ""           # REQUIRED – resource group that contains the APIM service

# ── APIM service ──────────────────────────────────────────────────────────────

apim_service_name = ""         # REQUIRED – name of the existing APIM service (e.g. apim-mycompany)

# ── MCP server ────────────────────────────────────────────────────────────────

mcp_server_url = ""            # REQUIRED – SERVICE_AGENT_URI from azd output
                               #            e.g. https://ca-agent-xxxx.azurecontainerapps.io

# ── Entra ID / JWT validation ─────────────────────────────────────────────────

tenant_id             = ""     # Leave empty to auto-detect from az account show
entra_api_app_client_id = ""   # REQUIRED – ENTRA_API_APP_CLIENT_ID from azd output

# ── APIM API definition ───────────────────────────────────────────────────────

api_name         = "mcp-text-utils-demo"   # Unique API identifier within the APIM service
api_display_name = "MCP Text Utilities"    # Display name in the developer portal
api_path         = "mcp-text-utils-demo"   # URL path suffix  (https://<apim>.azure-api.net/<api_path>)
subscription_required = True               # Require Ocp-Apim-Subscription-Key header

print("✅ Variables initialised")
print(f"   Resource Group : {resource_group or '(not set)'}")
print(f"   APIM Service   : {apim_service_name or '(not set)'}")
print(f"   MCP Server URL : {mcp_server_url or '(not set)'}")
print(f"   API Path Suffix: {api_path}")

<a id='1'></a>
### 1️⃣ Verify the Azure CLI and connected subscription

The cell below confirms that `az` is installed and that you are logged in.
If `subscription_id` or `tenant_id` were left empty above, they are populated automatically.

In [ ]:
import subprocess
import json
import os


def run_az(args, success_msg="", error_msg=""):
    """Run an az CLI command (as a list of arguments), print status, return parsed JSON."""
    print(f"⚙️  Running: {' '.join(args)}")
    result = subprocess.run(args, capture_output=True, text=True)
    if result.returncode != 0:
        msg = error_msg or "Command failed"
        print(f"❌ {msg}")
        print(result.stderr.strip())
        return None
    msg = success_msg or "OK"
    print(f"✅ {msg}")
    if result.stdout.strip():
        try:
            return json.loads(result.stdout)
        except json.JSONDecodeError:
            return result.stdout.strip()
    return {}


# ── Check current az account ──────────────────────────────────────────────────
account = run_az(["az", "account", "show"], "Retrieved az account", "Failed to get current account")

if not account:
    raise RuntimeError("Azure CLI login required. Run 'az login' and then re-run this cell.")

# Auto-populate empty variables
if not subscription_id:
    subscription_id = account["id"]
if not tenant_id:
    tenant_id = account["tenantId"]

print(f"👉  Current user   : {account['user']['name']}")
print(f"👉  Tenant ID      : {tenant_id}")
print(f"👉  Subscription ID: {subscription_id}")

# ── Basic validation ──────────────────────────────────────────────────────────
missing = [name for name, val in [
    ("resource_group", resource_group),
    ("apim_service_name", apim_service_name),
    ("mcp_server_url", mcp_server_url),
    ("entra_api_app_client_id", entra_api_app_client_id),
] if not val]

if missing:
    raise RuntimeError(f"Please set the following required variables in cell 0: {missing}")

print("\n✅ All required variables are set — ready to deploy.")

<a id='2'></a>
### 2️⃣ Deploy the MCP API to APIM

Runs `az deployment group create` using `apim-mcp.bicep`.
The Bicep template is compiled on the fly by the Azure CLI — no separate `az bicep build` step needed.

> **Note:** The deployment is idempotent. Re-running this cell updates existing resources.

In [ ]:
# Locate apim-mcp.bicep — expected to be in the same directory as this notebook
notebook_dir = os.path.dirname(os.path.abspath("deploy-mcp-to-apim.ipynb"))
bicep_path = os.path.join(notebook_dir, "apim-mcp.bicep")

if not os.path.exists(bicep_path):
    # Fallback: check the current working directory
    bicep_path = os.path.abspath("apim-mcp.bicep")

if not os.path.exists(bicep_path):
    raise RuntimeError(
        "apim-mcp.bicep not found. Run this notebook from the apim-deployment/ directory."
    )

# Change into the apim-deployment/ directory so loadTextContent('./apim-policy.xml')
# in the Bicep resolves correctly relative to the template file.
os.chdir(os.path.dirname(bicep_path))
print(f"📂 Working directory: {os.getcwd()}")

# ── Build the deployment command as a list (avoids shell injection) ───────────
sub_required_str = "true" if subscription_required else "false"

deploy_args = [
    "az", "deployment", "group", "create",
    "--subscription", subscription_id,
    "--resource-group", resource_group,
    "--template-file", "apim-mcp.bicep",
    "--parameters",
    f"apimServiceName={apim_service_name}",
    f"mcpServerUrl={mcp_server_url}",
    f"tenantId={tenant_id}",
    f"entraApiAppClientId={entra_api_app_client_id}",
    f"apiName={api_name}",
    f"apiDisplayName={api_display_name}",
    f"apiPath={api_path}",
    f"subscriptionRequired={sub_required_str}",
]

print("🚀 Deploying MCP API to APIM...")
print(f"   Template       : apim-mcp.bicep")
print(f"   Resource Group : {resource_group}")
print(f"   APIM Service   : {apim_service_name}")
print(f"   API Path Suffix: {api_path}")
print()

deployment = run_az(deploy_args, "Deployment completed successfully", "Deployment failed")

if deployment:
    outputs = deployment.get("properties", {}).get("outputs", {})
    if outputs:
        print()
        print("📤 Deployment outputs:")
        for key, val in outputs.items():
            print(f"   {key}: {val.get('value', '')}")

<a id='3'></a>
### 3️⃣ Verify the deployment

Retrieves the newly created API from APIM and prints the endpoint URLs.

In [ ]:
# Retrieve the APIM gateway URL
apim_info = run_az(
    [
        "az", "apim", "show",
        "--name", apim_service_name,
        "--resource-group", resource_group,
        "--subscription", subscription_id,
        "--output", "json",
    ],
    "Retrieved APIM service info",
    "Failed to retrieve APIM service info",
)

if apim_info:
    gateway_url = apim_info.get("gatewayUrl", "").rstrip("/")
    mcp_base_url = f"{gateway_url}/{api_path}"

    print()
    print("🌐 MCP API endpoints on APIM:")
    print(f"   MCP streamable-HTTP    : {mcp_base_url}/mcp")
    print(f"   SSE connection         : {mcp_base_url}/sse")
    print(f"   SSE messages           : {mcp_base_url}/messages/")
    print()
    print("📋 Environment variables for testing:")
    print(f"   export APIM_MCP_BASE_URL=\"{mcp_base_url}\"")

    if subscription_required:
        print()
        print("💡 To obtain a subscription key, go to:")
        print(f"   Azure Portal → {apim_service_name} → Subscriptions → Add subscription")
        print("   or use: az apim subscription list ...")